In [ ]:
def quantizeInput(X,BX):
    X = np.minimum(X,1.0-np.power(2.0,-BX))
    Xbs = []
    for i in range(BX):
        Xbi = np.greater_equal(X,0.5).astype(float)
        Xbs.append(Xbi)
        X = 2.0*X - Xbi
    carry = np.greater_equal(X,0.5).astype(float)
    for i in range(BX):
        j=BX-1-i
        Xbs[j] = Xbs[j]+carry
        carry = np.greater(Xbs[j],1.5).astype(float)
        Xbs[j] = Xbs[j]*np.not_equal(Xbs[j],2.0)
    return Xbs
def reconstructInput(Xbs,BX):
    X=np.zeros_like(Xbs[0])
    for l in range(BX):
        multiplier = np.power(0.5,l+1.0)
        X+=Xbs[l]*multiplier
    return X

def quantizeWeight(W,BW):
    W = np.minimum(W,1.0-np.power(2.0,-(BW-1.0)))
    Wbs = []
    Wbi = np.less(W,0).astype(float)
    Wbs.append(Wbi)
    W = (W + Wbi)
    for i in range(BW-1):
        Wbi = np.greater_equal(W,0.5).astype(float)
        Wbs.append(Wbi)
        W = 2.0*W - Wbi
    carry = np.greater_equal(W,0.5).astype(float)
    for i in range(BW):#-1):
        j=BW-1-i
        Wbs[j] = Wbs[j]+carry
        carry = np.greater(Wbs[j],1.5).astype(float)
        Wbs[j] = Wbs[j]*np.not_equal(Wbs[j],2.0)
    return Wbs

def reconstructWeight(Wbs,BW):
    W=np.zeros_like(Wbs[0])
    for j in range(BW):
        multiplier = np.power(0.5,j)
        if (j==0):
            multiplier=-1.0
        W+=Wbs[j]*multiplier
    return W

class DIMALinear(nn.Linear):

    def quantize_activations(self,input):
        input = reconstructInput(quantizeInput(input,BX))
        return input
               
    def quantize_weights(self):
        weight = self.weight.data.numpy()
        weight_q = quantizeWeight(weight,BW)
        for b in range(BW-1):
            weight_q[b+1] = weight_q[b+1]*(1+np.random.normal(0,sigma_D,weight_q[b+1].shape))
        weight = reconstructWeight(weight_q)
        Wmax = kclip*np.power(2.0,-(BW-1))
        weight = np.clip(weight,-Wmax,Wmax)
        self.weight = nn.Parameter(torch.from_numpy(weight))
        
    def forward(self, input: Tensor) -> Tensor:
        self.quantize_weights()
        input = self.quantize_activations(input)
#         if(list(self.weight.size)[0,:] > 256)
        return F.linear(input, self.weight, self.bias)